In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from os import wal
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Imports installations etc

In [2]:
# # !conda install -c pytorch pytorch-cpu torchvision
# # !yes Y | conda install -c pytorch pytorch-cpu torchvision
# from fastai.data.external import untar_data
# !conda install -c pytorch pytorch-cpu torchvision --y

In [3]:
# !conda install -c fastai fastaiorpip install http://download.pytorch.org/whl/cpu/torch-1.0.0-cp36-cp36m-linux_x86_64.whl --y

In [4]:
# !pip install fastai

In [5]:
# from fastai.vision import *

In [6]:
# !pip install ffmpeg

In [7]:
# import fastai # conda install -c pytorch -c fastai fastai=1.0.61

In [8]:
# conda install -c fastai fastbook
# from fastbook import *
# fastbook.setup_book()
import fastai
from fastai import *
from fastai.data.block import DataBlock
from fastai.data.external import *
from fastai.data.transforms import *
from fastai.vision import *
from fastai.vision.augment import RandomResizedCrop, Resize
from fastai.vision.data import ImageDataLoaders, ImageBlock



Convert videos to images, so we can actually work with them

In [9]:
import cv2
import matplotlib.pyplot as plt

In [10]:
# https://medium.com/howtoai/video-classification-with-cnn-rnn-and-pytorch-abe2f9ee031
# https://github.com/PacktPublishing/PyTorch-Computer-Vision-Cookbook/blob/master/Chapter10/myutils.py

def get_frames(filename, n_frames=1):
    print(filename)
    frames = []
    v_cap = cv2.VideoCapture(filename)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_list = np.linspace(0, v_len - 1, n_frames + 1, dtype=np.int16)

    for fn in range(v_len):
        success, frame = v_cap.read()
        if success is False:
            continue
        if fn in frame_list:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    v_cap.release()
    return frames, v_len


def store_frames(frames, path2store):
    for ii, frame in enumerate(frames):
        plt.imshow(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        # if not os.path.exists(path2store):
        #     os.makedirs(path2store)
        print(path2store)
        path2img = (path2store + "-frame-" + str(ii) + ".jpg").replace("\\", "/")
        # path2img = os.path.join(path2store, "-frame-" + str(ii) + ".jpg").replace("\\", "/")
        print(path2img)
        cv2.imwrite(path2img, frame)


temp_video_path = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/video/train/0-green-blue-9.avi"
temp_image_path = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/image/train/0-green-blue-9"
temp_frames, temp_frame_len = get_frames(temp_video_path, 5)
store_frames(temp_frames, temp_image_path)

D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/video/train/0-green-blue-9.avi


In [11]:
def extract_images_from_directory(video_directory, image_directory, f_num):
    _, _, filenames = next(walk(video_directory))
    for file in filenames:
        full_path = os.path.join(video_directory, file).replace("\\", "/")
        frames, len = get_frames(full_path, f_num)
        #cut out the file extension
        filename = file.split(".")[0]
        image_path = os.path.join(image_directory, filename).replace("\\", "/")
        store_frames(frames, image_path)


video_train_root = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/video/train/"
video_valid_root = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/video/valid/"
image_train_root = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/image/train/"
image_valid_root = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/image/valid/"
image_root = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/image"

frames_per_video = 5

# extract_images_from_directory(video_train_root, image_train_root, frames_per_video)
# extract_images_from_directory(video_valid_root, image_valid_root, frames_per_video)

# for file in valid_filenames:
#     full_path = os.path.join(video_valid_root, file).replace("\\", "/")
#     frames, len = get_frames(full_path, frames_per_video)
#     #cut out the file extension
#     filename = file.split(".")[0]
#     image_path = os.path.join(image_valid_root, filename).replace("\\", "/")
#     store_frames(frames, image_path)




In [12]:


# file_root = "../data/seizure_data/images/train/"
# train_image_root = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/image/train/"

# store_frames(test_frames, train_image_root)

# i = 0
# for frame in test_frames :
#     plt.imshow(frame)
#     # file_root = "../data/seizure_data/images/train/"
#     file_root = "D:/_PROJECTS/epilepsy-extension/src/data/seizure_data/images/train/"
#     file_name = "test-" + str(i)
#     file_extension = ".bmp"
#     filepath = file_root + file_name + file_extension
#     # filepath = "C:/Users/Alina Work/Desktop/temp.bmp"
#     cv2.imwrite(filepath, frame)
#     i += i

Build dataset from frames

In [13]:
# https://www.dataspoof.info/post/video-classification-with-fastai-and-deep-learning/
# np.random.seed(42)
# dataset = ImageDataLoaders.from_folder(
#     image_root,
#     item_tfms=Resize(224)
# )

# train='train',
# valid='valid',
# valid_pct=0.2,
# ds_tfms=get_transforms(),
# size=224,
# num_workers=4

# .normalize(imagenet_stats)

# dataset.show_batch()

TypeError: 'NoneType' object is not iterable

In [ ]:
# dataset.show_batch(max_n=50)
# data.classes

In [ ]:
# dataset.train_ds.items[:]

# dataset.vocab

In [ ]:
# !python main.py --root_path ~/data --video_path ucf101_videos/jpg --annotation_path ucf101_01.json \
# --result_path results --dataset ucf101 --n_classes 101 --n_pretrain_classes 700 \
# --pretrain_path models/resnet-50-kinetics.pth --ft_begin_module fc \
# --model resnet --model_depth 50 --batch_size 128 --n_threads 4 --checkpoint 5

In [16]:
import torch
from torch import nn


def load_pretrained_model(model, pretrain_path, model_name, n_finetune_classes):
    if pretrain_path:
        print('loading pretrained model {}'.format(pretrain_path))
        pretrain = torch.load(pretrain_path, map_location='cpu')

        model.load_state_dict(pretrain['state_dict'])
        tmp_model = model
        if model_name == 'densenet':
            tmp_model.classifier = nn.Linear(tmp_model.classifier.in_features,
                                             n_finetune_classes)
        else:
            tmp_model.fc = nn.Linear(tmp_model.fc.in_features,
                                     n_finetune_classes)

    return model


pretrained_model = None
pretrained_model_path = "D:/_PROJECTS/epilepsy-extension/models/r3d50_KMS_200ep.pth"
pretrained_model = load_pretrained_model(pretrained_model, pretrained_model_path, "model name", 5)
print(pretrained_model)

loading pretrained model D:/_PROJECTS/epilepsy-extension/models/r3d50_KMS_200ep.pth


AttributeError: 'NoneType' object has no attribute 'load_state_dict'